In [1]:
import torch
import torch_geometric
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from tqdm.notebook import tqdm
import numpy as np
import os
import sys

root_file = "/home/dmngo/test-train-hep"

In [2]:
import yaml

with open(root_file + '/reg_defs.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    definitions = yaml.load(file, Loader=yaml.FullLoader)
    
features = definitions['features']
spectators = definitions['spectators']
labels = definitions['labels']

nfeatures = definitions['nfeatures']
nspectators = definitions['nspectators']
nlabels = definitions['nlabels']
ntracks = definitions['ntracks']

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import EdgeConv, global_mean_pool
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, BatchNorm1d, Conv1d
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer

In [4]:
from torch_geometric.data import Data, DataListLoader, Batch
from torch.utils.data import random_split
import os.path as osp
from GraphDataset import GraphDataset
import glob
import shutil

In [104]:
torch.tensor(1).to_np

AttributeError: 'Tensor' object has no attribute 'to_np'

In [5]:
from RegDataGenerator import DataGenerator
train_files = ['/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_1-1_Skim.root']
dir_path = '/home/dmngo/test-train-hep/data/gdata_train'
graph_dataset = GraphDataset(dir_path, features, labels, spectators, n_events=1000, n_events_merge=1, 
                             file_names= train_files)

In [6]:
train_generator = DataGenerator(train_files, features, labels, spectators, batch_size=1024, n_dim=ntracks, 
                                remove_mass_pt_window=False, 
                                remove_unlabeled=False, max_entry=20000)
X, y = train_generator[1]

In [7]:
def collate(items):
    l = sum(items, [])
    return Batch.from_data_list(l)

torch.manual_seed(0)
valid_frac = 0.20
full_length = len(graph_dataset)
valid_num = int(valid_frac*full_length)
batch_size = 32

train_dataset, valid_dataset = random_split(graph_dataset, [full_length-valid_num,valid_num])

train_loader = DataListLoader(train_dataset, batch_size=batch_size, pin_memory=True, shuffle=True)
train_loader.collate_fn = collate
valid_loader = DataListLoader(valid_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
valid_loader.collate_fn = collate

train_samples = len(train_dataset)
valid_samples = len(valid_dataset)

/opt/conda/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataListLoader' is deprecated, use 'loader.DataListLoader' instead
  warnings.warn(out)


In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer = Seq(Lin(19, 32),
                         BatchNorm1d(32),
                         ReLU(),
                         Lin(32,1))

    def forward(self, x, batch):
        out = scatter_mean(x, batch, dim=0)
        return self.layer(out)
    
net = Net().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.02)

In [9]:
n_epochs = 1
t = tqdm(range(0, n_epochs))

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
for epoch in t:
    t = tqdm(enumerate(train_loader), total=train_samples/batch_size, leave=bool(epoch==n_epochs-1))
    loss_func = nn.L1Loss()
    for i, data in t:
        data = data.to(device)
        y = data.y
        prediction = net(data.x, data.batch)
        print(data.x.shape)
        loss = loss_func(prediction.float(), y.float())
        print(prediction.float(), y.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  0%|          | 0/75.0 [00:00<?, ?it/s]

torch.Size([3200, 19])
tensor([[ 0.5470],
        [-0.6541],
        [-0.3746],
        [ 0.5952],
        [-0.4541],
        [-0.2666],
        [ 1.3029],
        [-0.4477],
        [ 1.0893],
        [ 0.3382],
        [ 0.7987],
        [ 1.1430],
        [-0.1173],
        [ 0.2319],
        [ 0.1558],
        [-0.0875],
        [ 1.1133],
        [ 0.1678],
        [-0.5018],
        [-0.4639],
        [-0.4980],
        [ 0.0509],
        [ 1.3796],
        [-0.1998],
        [-0.0643],
        [ 0.1346],
        [ 0.0415],
        [-0.5857],
        [-0.3505],
        [-0.2821],
        [ 0.7007],
        [-0.4231]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[172.],
        [  4.],
        [ 65.],
        [ 25.],
        [113.],
        [  5.],
        [  2.],
        [ 46.],
        [ 31.],
        [ 31.],
        [ 72.],
        [  2.],
        [ 17.],
        [244.],
        [299.],
        [ 33.],
        [315.],
        [ 48.],
        [ 11.],
        [294.],
      

torch.Size([3200, 19])
tensor([[1.0572],
        [1.5053],
        [3.5960],
        [1.0832],
        [4.2588],
        [4.1714],
        [1.7146],
        [1.2216],
        [3.6329],
        [3.5902],
        [0.7396],
        [2.2230],
        [1.5182],
        [1.5427],
        [4.1258],
        [3.1079],
        [0.6230],
        [0.9027],
        [6.8962],
        [3.9489],
        [5.2556],
        [0.6773],
        [2.8139],
        [1.2708],
        [3.4665],
        [0.8254],
        [3.9131],
        [1.4975],
        [2.2423],
        [4.4956],
        [3.8134],
        [1.9722]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[281.],
        [335.],
        [275.],
        [209.],
        [226.],
        [483.],
        [  4.],
        [ 93.],
        [  1.],
        [289.],
        [195.],
        [200.],
        [ 17.],
        [263.],
        [ 94.],
        [  7.],
        [313.],
        [  4.],
        [293.],
        [ 73.],
        [279.],
        [ 10.],
      

torch.Size([3200, 19])
tensor([[ 4.1574],
        [ 7.2519],
        [ 7.6030],
        [ 3.7725],
        [ 7.4021],
        [ 2.0566],
        [ 2.4949],
        [ 3.9900],
        [ 3.8146],
        [ 4.3818],
        [12.1260],
        [ 8.4804],
        [ 3.4794],
        [ 2.8889],
        [ 3.9533],
        [ 2.8784],
        [ 9.1804],
        [11.2699],
        [ 6.0827],
        [ 6.1893],
        [ 3.5253],
        [ 4.2535],
        [ 2.1143],
        [ 2.5062],
        [11.8166],
        [ 3.9689],
        [ 4.2899],
        [ 9.8238],
        [ 8.8923],
        [ 4.6178],
        [ 1.7909],
        [ 3.8610]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[ 79.],
        [  3.],
        [ 93.],
        [  5.],
        [ 70.],
        [ 40.],
        [  3.],
        [  4.],
        [  3.],
        [287.],
        [191.],
        [  5.],
        [  3.],
        [ 16.],
        [  1.],
        [ 68.],
        [ 10.],
        [  2.],
        [284.],
        [283.],
      

torch.Size([3200, 19])
tensor([[ 5.6294],
        [21.7175],
        [ 8.6207],
        [ 5.0970],
        [ 4.9669],
        [ 5.4303],
        [14.3772],
        [ 5.3019],
        [10.0253],
        [ 8.6113],
        [19.4475],
        [11.9700],
        [ 8.3765],
        [12.8562],
        [ 9.5309],
        [ 5.0009],
        [10.1806],
        [ 8.1011],
        [ 6.5905],
        [ 9.3826],
        [ 7.4517],
        [ 5.5407],
        [ 7.6472],
        [11.5174],
        [19.2705],
        [ 4.2230],
        [ 8.4223],
        [ 7.7434],
        [ 5.8348],
        [ 8.5620],
        [16.3079],
        [14.4895]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[ 54.],
        [165.],
        [  8.],
        [368.],
        [ 52.],
        [ 13.],
        [ 16.],
        [ 31.],
        [284.],
        [ 53.],
        [  8.],
        [  5.],
        [ 72.],
        [222.],
        [ 32.],
        [  3.],
        [ 29.],
        [  2.],
        [ 74.],
        [132.],
      

...

torch.Size([3200, 19])
tensor([[41.9045],
        [17.9910],
        [31.7594],
        [19.3938],
        [19.1977],
        [19.6064],
        [24.6621],
        [18.9461],
        [20.4600],
        [49.7937],
        [20.7339],
        [25.3950],
        [54.8228],
        [22.5743],
        [20.8556],
        [17.9338],
        [40.4301],
        [35.5572],
        [21.2986],
        [33.2846],
        [47.4443],
        [20.2765],
        [19.4743],
        [19.1088],
        [28.6928],
        [35.9367],
        [54.4027],
        [20.0271],
        [18.7795],
        [18.6337],
        [17.0878],
        [47.7229]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[196.],
        [  5.],
        [  1.],
        [164.],
        [ 14.],
        [  6.],
        [ 12.],
        [  5.],
        [ 90.],
        [140.],
        [ 34.],
        [ 37.],
        [ 67.],
        [234.],
        [269.],
        [  1.],
        [ 10.],
        [160.],
        [189.],
        [167.],
      

torch.Size([3200, 19])
tensor([[22.1833],
        [25.4912],
        [22.7523],
        [26.0113],
        [28.1441],
        [34.6881],
        [24.7523],
        [55.9528],
        [30.1602],
        [30.4674],
        [27.0965],
        [26.0411],
        [38.7116],
        [23.5573],
        [24.7919],
        [27.7916],
        [49.2369],
        [27.5135],
        [26.9075],
        [64.9608],
        [29.5476],
        [25.5713],
        [32.0711],
        [40.1654],
        [50.8665],
        [30.3076],
        [24.4729],
        [45.1981],
        [67.1439],
        [32.8026],
        [24.8170],
        [60.2807]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[  3.],
        [ 91.],
        [192.],
        [ 11.],
        [ 49.],
        [ 67.],
        [ 84.],
        [ 11.],
        [ 12.],
        [  3.],
        [  4.],
        [163.],
        [ 88.],
        [ 24.],
        [115.],
        [440.],
        [  4.],
        [ 11.],
        [  5.],
        [ 34.],
      

torch.Size([3200, 19])
tensor([[48.7267],
        [39.9921],
        [31.8323],
        [32.3420],
        [39.0004],
        [27.8132],
        [29.8818],
        [31.0874],
        [37.2854],
        [40.3381],
        [45.5787],
        [32.1372],
        [29.3316],
        [67.5946],
        [30.5160],
        [40.9419],
        [39.9145],
        [60.7043],
        [31.0698],
        [61.5881],
        [29.0269],
        [32.6084],
        [28.8187],
        [28.5131],
        [30.7181],
        [30.1688],
        [43.5117],
        [61.6989],
        [59.0724],
        [47.0609],
        [59.2354],
        [61.5780]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[ 21.],
        [340.],
        [  2.],
        [  3.],
        [ 18.],
        [  2.],
        [ 32.],
        [ 20.],
        [137.],
        [  6.],
        [ 11.],
        [  4.],
        [ 46.],
        [  3.],
        [ 72.],
        [341.],
        [ 82.],
        [ 22.],
        [  2.],
        [119.],
      

torch.Size([3200, 19])
tensor([[41.1272],
        [34.4884],
        [35.0597],
        [36.2539],
        [57.3766],
        [36.1574],
        [61.2408],
        [34.2830],
        [34.3230],
        [47.9811],
        [38.1595],
        [31.4066],
        [34.7335],
        [50.1636],
        [35.0070],
        [39.9223],
        [35.1213],
        [52.7507],
        [49.7222],
        [38.8398],
        [39.2354],
        [64.5244],
        [33.7922],
        [70.3736],
        [43.5973],
        [38.4956],
        [86.5642],
        [43.2812],
        [34.4771],
        [39.7765],
        [59.6817],
        [50.4768]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[191.],
        [ 16.],
        [ 65.],
        [196.],
        [ 73.],
        [105.],
        [330.],
        [  1.],
        [190.],
        [ 18.],
        [  1.],
        [ 90.],
        [279.],
        [  7.],
        [179.],
        [204.],
        [267.],
        [295.],
        [ 83.],
        [151.],
      

torch.Size([3200, 19])
tensor([[ 37.2378],
        [ 47.2947],
        [ 40.1770],
        [ 52.5994],
        [ 37.4485],
        [ 39.6022],
        [ 44.8247],
        [ 42.1348],
        [ 38.8514],
        [ 42.0864],
        [ 81.2282],
        [ 57.2750],
        [ 47.2038],
        [ 38.2554],
        [ 67.2391],
        [ 38.4898],
        [113.9187],
        [ 37.7470],
        [ 51.8569],
        [ 51.7051],
        [ 47.9840],
        [ 38.5819],
        [ 41.5190],
        [ 66.1039],
        [ 37.4059],
        [ 38.4053],
        [ 65.1517],
        [ 42.8295],
        [ 36.0294],
        [ 38.7246],
        [ 38.7666],
        [ 37.7596]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[ 19.],
        [ 37.],
        [210.],
        [ 81.],
        [ 48.],
        [ 10.],
        [  5.],
        [  5.],
        [  7.],
        [ 13.],
        [101.],
        [249.],
        [ 71.],
        [281.],
        [448.],
        [  5.],
        [ 87.],
        [ 63.],
      

In [11]:
inputs = 19
hidden = 40
outputs = 1

class Edge(torch.nn.Module):
    def __init__:
    def forward():
        
class NodeBlock(torch.nn.Module):
    def __init__:
    def forward():
        
class GlobalBlock(torch.nn.Module):
    def __init__:
    def forward():
        
class InteractionNetwork(torch.nn.Module):
    def __init__:
    def forward():
        

SyntaxError: invalid syntax (<ipython-input-11-fc081e70dc5f>, line 6)

In [12]:
test_files = ["/home/dmngo/teams/DSC180A_FA21_A00/a11/test_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_3-10_Skim.root"]
#delete old processed files
dir_path = '/home/dmngo/test-train-hep/src/data/gdata_test'
try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))
test_dataset = GraphDataset(dir_path, features, labels, spectators, n_events=2000, n_events_merge=1, 
                            file_names=test_files)

test_loader = DataListLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
test_loader.collate_fn = collate
test_samples = len(test_dataset)

Processing...


  0%|          | 0/2000 [00:00<?, ?it/s]

Done!


In [14]:
y_test = []
y_predict = []
t = tqdm(enumerate(test_loader), total=train_samples/batch_size, leave=bool(epoch==n_epochs-1))
for i,data in t:
    data = data.to(device)    
    batch_output = net(data.x, data.batch)    
    y_predict.append(batch_output.detach().cpu().numpy())
    y_test.append(data.y.cpu().numpy())
y_test = np.concatenate(y_test)
y_predict = np.concatenate(y_predict)

  0%|          | 0/75.0 [00:00<?, ?it/s]

In [20]:
#rmse
np.sqrt(np.mean((y_test - y_predict) ** 2))

273.97795594255837

array([[39.4381  ],
       [44.077225],
       [73.250244],
       ...,
       [39.076866],
       [40.311817],
       [43.20022 ]], dtype=float32)

In [21]:
import os.path as osp
modpath = osp.join('simplenetwork_best.pth')
print('New best model saved to:',modpath)
torch.save(net.state_dict(),modpath)

New best model saved to: simplenetwork_best.pth
